In [119]:
import pandas as pd
import numpy as np
import requests
import os
import glob
import shutil
import string

In [73]:
def calc_target(kpis):
    return sum(5*(x_ref - x_ours) if x_ref > x_ours else x_ref - x_ours for x_ref, x_ours in kpis)

In [132]:
# Helper functions

def getAmount(solution):
    solution2 = solution.loc[~(solution.drop(columns="amount") == 0).all(axis=1)]
    return solution2["amount"].mean() * 1


def getVariance(solution):
    solution_amountless = solution.drop(columns="amount")
    temp = {col: 0 for col in solution_amountless.columns}
    for index, nominator in solution.iterrows():
        nominator_amountless = nominator.drop(columns="amount")
        selected = nominator_amountless[nominator_amountless==1].index.tolist()

        if len(selected) == 0: continue

        amount_to_distribute = nominator["amount"] / len(selected)

        for validator in selected:
            temp[validator] += amount_to_distribute
            
    return np.array(list(temp.values())).std() * -1


def getAssignment(solution):
    nominatorsAssignments = (solution.iloc[:, :-1] == 1).sum(axis=1)
    return ((nominatorsAssignments - 1) ** 2).sum() * -1


def concatenateSolutions(sol1, sol2):
    result = []
    for s1, s2 in zip(sol1, sol2):
        normalizer = max(s1, s2)
        result.append((s1 / normalizer, s2 / normalizer))

    return result


def calculateKpis(sol):
    solution_amount = getAmount(sol)
    solution_variance = getVariance(sol)
    solution_assignment = getAssignment(sol)

    solution_kpis = [solution_amount, solution_variance, solution_assignment]

    return solution_kpis


def compareSolutions(sol1, sol2):
    return calc_target(concatenateSolutions(calculateKpis(sol1), calculateKpis(sol2)))


def getDataBatches(nominators_filepath_pattern='data/polkadot_nominators_session_*.csv', batch_size=32):
    file_paths = glob.glob(nominators_filepath_pattern)
    num_batches = len(file_paths) // batch_size + (1 if len(file_paths) % batch_size != 0 else 0)
    data_batches = []
    for i in range(num_batches):
        nominators_batches = [pd.read_csv(file) for file in file_paths[i * batch_size: (i + 1) * batch_size]]
        all_data = pd.concat(nominators_batches, ignore_index=True)
        min_batch_amount = all_data["bonded_amount"].min()
        max_batch_amount = all_data["bonded_amount"].max()

        nominators_batches = list(map(lambda x: normalizeAmountColumn(x, min_batch_amount, max_batch_amount), nominators_batches))
        data_batches.append(nominators_batches)
    return data_batches


def normalizeAmountColumn(nominators_df, min_amount, max_amount):
    nominators_df['bonded_amount'] = (nominators_df['bonded_amount'] - min_amount) / (max_amount - min_amount)
    return nominators_df

In [135]:
getDataBatches(batch_size=4)[0][0]

,era,session,block_number,stash_address,controller_address,bonded_amount,num_targets,targets
0,191,1187,2838905,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,6.684753e-05,1,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ
1,191,1187,2838905,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,1.834434e-06,16,1zugcacYFxX3HveFpJVUShjfb3KyaomfVqMTFoxYuUWCdD...
2,191,1187,2838905,15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,15wtWHaQemsJL4uPKGKCRwnpaGUK66VdLjd8wJKWcpoEtetf,6.134484e-05,16,1zugcabYjgfQdMLC3cAzQ8tJZMo45tMnGpivpAzpxB4CZy...
3,191,1187,2838905,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,4.873435e-03,5,14cxMDpBNLsNEXWyCzked3zghzaYWXwoqGT4h12GqQXdVh...
4,191,1187,2838905,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,6.592261e-06,16,14SRtmkZPGzaV1bKivd7T6xw15YRW53gjE5F2LVbiwChqK...
...,...,...,...,...,...,...,...,...
8951,191,1187,2838905,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,4.663159e-06,10,15awXd39ZEttFj5tyM6Lkq5ReefCNcrG8f2agSgn2ZCjoJ...
8952,191,1187,2838905,169xNhgMQ8aoqTq6mi4mME7GDRxH179rSJBNjGhV3q3VM4rr,169xNhgMQ8aoqTq6mi4mME7GDRxH179rSJBNjGhV3q3VM4rr,4.875600e-07,4,14ghKTz5mjZPgGYvgVC9VnFw1HYZmmsnYvSSHFgFTJfMvw...
8953,191,1187,2838905,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,4.161369e-07,16,12vCVEpCz81rvS23PEJQ6KBDbPWgvyVX5Q1PrgQbVvLc4i...
8954,191,1187,2838905,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,2.012874e-04,5,15DLJZ4ceN58vEgDiQjK8JsSJuLNBqhUnQ6QCY1QNSjrQn...


In [75]:
google_cloud_url = "https://storage.googleapis.com/watcher-csv-exporter/"
session_filename_template = string.Template("polkadot_nominators_session_$id.csv")
era_filename_template = string.Template("polkadot_validators_era_$id.csv")

Loading of data starting from era number 165 and session 1031

In [76]:
def download_file(url, destination):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(destination, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

def download_batch(starting_era=165, starting_session=1031, destinationFolder="data/", number_of_eras_to_download=100):
    if os.path.exists(destinationFolder):
         shutil.rmtree(destinationFolder)
         
    os.makedirs(destinationFolder, exist_ok=True)
    era_id = starting_era
    session_id = starting_session
    total_eras_downloaded = 0

    while total_eras_downloaded < number_of_eras_to_download:
        era_filename = era_filename_template.substitute({'id': era_id})

        try:
            download_file(
                google_cloud_url + era_filename,
                destinationFolder + era_filename
            )
        except Exception as e:
            era_id += 1
            session_id += 6
            continue
        
        session_filename = session_filename_template.substitute({'id': session_id})
        download_file(
            google_cloud_url + session_filename,
            destinationFolder + session_filename
        )

        total_eras_downloaded += 1
        era_id += 1
        session_id += 6

download_batch()

In [77]:
nominators = pd.read_csv("data/polkadot_nominators_session_1031.csv")
validators = pd.read_csv("data/polkadot_validators_era_165.csv")

In [78]:
nominators

,era,session,block_number,stash_address,controller_address,bonded_amount,num_targets,targets
0,165,1031,2465718,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,39498643450307,1,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ
1,165,1031,2465718,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,1084592814654,16,1zugcacYFxX3HveFpJVUShjfb3KyaomfVqMTFoxYuUWCdD...
2,165,1031,2465718,15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,15wtWHaQemsJL4uPKGKCRwnpaGUK66VdLjd8wJKWcpoEtetf,36219110590042,16,1zugcabYjgfQdMLC3cAzQ8tJZMo45tMnGpivpAzpxB4CZy...
3,165,1031,2465718,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,2352067003888369,5,14cxMDpBNLsNEXWyCzked3zghzaYWXwoqGT4h12GqQXdVh...
4,165,1031,2465718,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,3894263543601,16,14SRtmkZPGzaV1bKivd7T6xw15YRW53gjE5F2LVbiwChqK...
...,...,...,...,...,...,...,...,...
8198,165,1031,2465718,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,2780000000000,10,15awXd39ZEttFj5tyM6Lkq5ReefCNcrG8f2agSgn2ZCjoJ...
8199,165,1031,2465718,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,245783340884,16,12vCVEpCz81rvS23PEJQ6KBDbPWgvyVX5Q1PrgQbVvLc4i...
8200,165,1031,2465718,14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,14529640,4,12LKeuFyyjC94iXpHftt3UVu567ji5WyKE6MvDBDWVJUuu...
8201,165,1031,2465718,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,13gfodUESWspVuk5xHdkC8uCJi3EAvTtjiaRCUsN9ktwhpKR,120000000000000,5,15DLJZ4ceN58vEgDiQjK8JsSJuLNBqhUnQ6QCY1QNSjrQn...


In [79]:
number_of_validators = len(validators)
validators

,era,session,block_number,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,voters,era_points
0,165,1031,2465718,Wei,14ShUZUYUR35RBZW6uVVt1zXDxmSQddkeDdXf1JkMA6P721N,16knCdWHWokjeciJuassAub21PHi3YTt94um4DjRJSRtF7sh,1.0,2331305558010386,30622799978313560,127,1831,1260
1,165,1031,2465718,NaN,1FCu68ZwBHNzZLcGa92eHwvR61hk3MpjrhiqN96xF9vWS1Q,13gAjcC56upxCdaNS69jB7rvGDzimcztTm9Jxhfgma8w9jor,100.0,99360406100,37263464975913580,2,27,1300
2,165,1031,2465718,Zug Cap / 19,1zugcag7cJVBtVRnFxv5Qftn7xKAnR6YJ9x4x3XLgGgmNnS,13eeBGJPSPQjXGfBmd2rsek8gKU1ivz7eZReiVRwwES1QzuX,0.7,100000000000,30549484370550930,106,1203,1260
3,165,1031,2465718,Polkadot.pro - Realgar,1REAJ1k691g5Eqqg9gL7vvZCBG7FCCZ8zgQkZWd4va5ESih,1ZMbuCR3QiatxRsQdNnJYgydn3CWV4PELcTzpH4TNoNjxno,1.0,5342981819844241,34321621635570840,128,1329,1120
4,165,1031,2465718,Able Wanderer / 🐁 Mouse-6,135wKSGZpQcTrhmFju9VeKC68gMBMsCYpSNtDyoqj3BVSZ1R,135wKSGZpQcTrhmFju9VeKC68gMBMsCYpSNtDyoqj3BVSZ1R,100.0,10000000000,26135922035705640,4,9,1180
...,...,...,...,...,...,...,...,...,...,...,...,...
223,165,1031,2465718,NaN,14yVcx8AtnmaZMCZWVp8AKDgnAYdjeQjAbPL1HbvPxr74Fsf,16H1vG9t6EzS64r4YgLHBEXwKkVG23gE8t84DzXTtUi1ucxv,100.0,99360406100,24840579921059186,1,1,1200
224,165,1031,2465718,P2P.ORG / 18,16Sud9b5uUfUi1HXdfwb3drbYBZBLPVvdKuZhwxz2n7HR12M,16Sud9b5uUfUi1HXdfwb3drbYBZBLPVvdKuZhwxz2n7HR12M,3.0,100000533506,30626119242918828,50,283,1280
225,165,1031,2465718,🔒stateless_money🔒,12713bbq45c66CN9AD7yusSXWE1kY91DcMpjVcB2rXqZKy2w,1f2K1YccCZEfbcGrHshQWzrGdXp6bxFgMahGrKYZm9HPuy1,1.0,2000590510261,31150322561480132,15,95,1220
226,165,1031,2465718,NaN,15rQD4ykz4uRD7YGdTpMT8UmgkNDJRKckZbAocALndBVzCjf,15rQD4ykz4uRD7YGdTpMT8UmgkNDJRKckZbAocALndBVzCjf,100.0,10000000000,24620024373998627,3,7,1020


In [80]:
target = nominators["targets"][0]
validators[validators["stash_address"] == target]

,era,session,block_number,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,voters,era_points
97,165,1031,2465718,Listen,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ,136JdPA8uidFAUPJKarwpBsPqGkkRqLTYgNBC7vuwyAnLd6a,5.0,1799881215889431,30614677019378986,171,550,1580


Creation of binary table similar to one in presentation, so showing which validators nominator selected and his total money

In [81]:
binary_matrix = nominators["targets"].str.get_dummies(sep=',')
binary_matrix.index = nominators["stash_address"]
binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]
binary_matrix

,111111111111111111111111111111111HC1,1124RsfEgJEZvEq4HbtGFcpqoxnqSy79EjNZY9tzPct3AB6o,112Ji3ASaU21FGa4ymrjvyZrDv8LbW9tF5NLUksdLdJkjY7E,1155dDdp1X4F3rh35hAMoK8r4iDVdzprtRpVrScP35YPC2b,11BgR7fH8Sq6CcGcXxZrhyrBM2PUpDmhnGZpxPGvVGXEiPT,11MJU5Q1rQh5BKuuECePhSAutv3WEVx6f2x9eZk9HXkCC1e,11gqpAyU17G9EFW5n5MNngh824F3Y2as72V2rgc7Wq5JVRd,11uMPbeaEDJhUxzU4ZfWW9VQEsryP9XqFcNRfPdYda6aFWJ,121GBc2ZK53Uoc6JovJjU7zARH8fys5efyUfHKT9vABpqajx,121ZiNk5DKVKUuYQtuNcHC25AD2K8bSNaaFn7qSa4JrSwYBR,...,1zugcauBVvcNa6tpW3j6WaGRL1Wvm9oSimK2eP3VNqNLw1V,1zugcavJYzi2KErZy9CMbLANhfrFwMESgPz9q29eUCR5gTW,1zugcavYA9yCuYwiEYeMHNJm9gXznYjNfXQjZsZukF1Mpow,1zugcawsx74AgoC4wz2dMEVFVDNo7rVuTRjZMnfNp9T49po,1zugcaxRrQDr7ktb6SpjVMkywys2ysoZWBfUfs9CDPwFVHC,1zugcaxcGLmRb6wFpftx99sYRSQihqq6KLTThimEkYsaSoq,1zugcazy9vaQJkMX79DYwFdvTYGmAbjPQrYuyUNHoFnXKtA,1zugcdarkTpYvG69VFJvxCdMVovSoWPieafJgLCnWCsYCzS,1zugchKTRDgvvfeGGSf5KpVWMx17Atf956FJYaFrmxdS4T7,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39498643450307
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1084592814654
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,36219110590042
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2352067003888369
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3894263543601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2780000000000
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,245783340884
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14529640


Creation of table as above, but with number of how many specific validators nominator selected instead of information if validator is present in targets of nominator.

In [82]:
unique_validators = pd.Series(nominators["targets"].str.split(",").sum()).unique()
count_matrix_np = np.zeros((len(nominators), len(unique_validators)), dtype=int)
validator_to_idx = {validator: i for i, validator in enumerate(unique_validators)}

for i, targets in enumerate(nominators['targets']):
    for validator in targets.split(','):
        count_matrix_np[i, validator_to_idx[validator]] += 1

count_matrix = pd.DataFrame(count_matrix_np, index=nominators['stash_address'], columns=unique_validators)
count_matrix['amount'] = nominators.set_index('stash_address')['bonded_amount']
count_matrix

,1hJdgnAPSjfuHZFHzcorPnFvekSHihK9jdNPWHXgeuL7zaJ,1zugcacYFxX3HveFpJVUShjfb3KyaomfVqMTFoxYuUWCdD8,1zugcaebzKgKLebGSQvtxpmPGCZLFoEVu6AfqwD7W5ZKQZt,1zugcakrhr3ZR7q7B8WKuaZY5BjZAU43m79xEyhNQwLTFjb,1zugcarJnZ4ft2PiJoGg6DgmZjnKNBrcKTFrAzhGPCX6bJ5,1zugcaxRrQDr7ktb6SpjVMkywys2ysoZWBfUfs9CDPwFVHC,1zugcabTuN7rs1bFYb33gRemtg67i4Mvp1twW85nQKiwhwQ,1zugcaj4mBMu7EULN4rafT5UTfBjbvqaoypZyxWa3io6qJS,14ShUZUYUR35RBZW6uVVt1zXDxmSQddkeDdXf1JkMA6P721N,16GDRhRYxk42paoK6TfHAqWej8PdDDUwdDazjv4bAn4KGNeb,...,15YRqDv5mYuVoeFcW7Y2NWgB5Ut6bc9C8y35yAC7aT6P3YzD,15BTJ9g5Rqm9PxHXh6sqMM5QVDXpwtQCTHp5MvbHbxnkvivy,13iXZVPpXuRnrSmKsqM51NtR8nqD5arfovegQT75r3Pp1Hpe,1zugca4bLxr1BB6gngcKaZmV8u76tmAJagg6FhePZjNJmam,13b7JdEzrXvGbkACdrbCXayeMnbcYT5NMK6paWdmM9328xsw,15r3BYwQ7ucMirgWuaZt5L1GxJt7JxbDtUWq8S8381B8LqZf,135zW8niMZUNGZQ7e8V1wSXzyaicfq1rAgpXugzxQkrHr1Mn,14JZK1nqHpnZ1imkpv6T5RMGKoCbjg2Q7zZ2omDimFgMvrzk,12GTt3pfM3SjTU6UL6dQ3SMgMSvdw94PnRoF6osU6hPvxbUZ,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39498643450307
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1084592814654
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,36219110590042
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2352067003888369
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3894263543601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2780000000000
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,245783340884
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,14529640


In [83]:
binary_matrix_dropped = binary_matrix.drop(columns="amount")
random_solution = binary_matrix_dropped.sample(n=number_of_validators, axis=1)
random_solution

,1zugcam9xSdgVtn4wDtFbvrRRnvW7r5mCCBDZeNEaLNiQB9,124YFXA3XoRs9Epcx3aRUSk3EKYaznocqMWfrMKtGjx8TJ2W,15MUBwP6dyVw5CXF9PjSSv7SdXQuDSwjX86v1kBodCSWVR7c,16ZzMovSVVLU5oP2o5PwNG2ybbdT2diiKPAsz6J37myEGsDw,1zugcagpF2GXENgfYE1sL5fTSscZq1s5MA4znWdRcpEF4yw,15KDFYfFjdqhp3MDFEtHuyu9kLpXbT7k1zjx78MphViFdCaU,1341kcAXqyA5ZJHBdPoLeHsDtmqLBZ2qm5Hu7e9tDR9Jv5RM,1zugcaSGKfFvwP4cnySCddnwpr3xDvWiAEmQLbQ8NxEDVRV,153uHgQYh3DxFKNKVqRuoftD4pmmAiUvw41p1UgUstLnKLwh,1R2BJXTDefF5gi2CiTv4qkdtfprC9TChtfZqamEmkHa5c7d,...,1vEVWfqoLErB6MhhtDijrnmnHqjhrrFA5GzXGNL2HwESQ5r,16WU9RRW2XjnfzrdMJa2JHmXfeCyWGRayWHNQCT8r16KnrQd,1zugcaaaDTLhG77kp7PBPpWiaUWTND9oKNcNM94StNStnuw,14cxMDpBNLsNEXWyCzked3zghzaYWXwoqGT4h12GqQXdVhmn,16idW666JZya7ZrGcL2nqSRZLRbQY3Tnf8HorbpMCeG83uq,19z6EGB9qgN7MsQACNYCmotAVb8FNL11JiXg7FV1hs6w3nc,13asdY4e7sWdJ4hbGW9n2rkNro1mx5YKB6WBCC9gvqKmLvNH,1qFBkRovtyKwWznGdQrA2qvmpndhM513TPoWf6f4oTVReLs,14N5nJ4oR4Wj36DsBcPLh1JqjvrM2Uf23No2yc2ojjCvSC24,1ejsVb4xGv2kVGeM2x9iGxJdvLE9QMn8Nh8dhMv8cHCHnTL
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
random_solution['amount'] = nominators.set_index('stash_address')['bonded_amount']
random_solution

,1zugcam9xSdgVtn4wDtFbvrRRnvW7r5mCCBDZeNEaLNiQB9,124YFXA3XoRs9Epcx3aRUSk3EKYaznocqMWfrMKtGjx8TJ2W,15MUBwP6dyVw5CXF9PjSSv7SdXQuDSwjX86v1kBodCSWVR7c,16ZzMovSVVLU5oP2o5PwNG2ybbdT2diiKPAsz6J37myEGsDw,1zugcagpF2GXENgfYE1sL5fTSscZq1s5MA4znWdRcpEF4yw,15KDFYfFjdqhp3MDFEtHuyu9kLpXbT7k1zjx78MphViFdCaU,1341kcAXqyA5ZJHBdPoLeHsDtmqLBZ2qm5Hu7e9tDR9Jv5RM,1zugcaSGKfFvwP4cnySCddnwpr3xDvWiAEmQLbQ8NxEDVRV,153uHgQYh3DxFKNKVqRuoftD4pmmAiUvw41p1UgUstLnKLwh,1R2BJXTDefF5gi2CiTv4qkdtfprC9TChtfZqamEmkHa5c7d,...,16WU9RRW2XjnfzrdMJa2JHmXfeCyWGRayWHNQCT8r16KnrQd,1zugcaaaDTLhG77kp7PBPpWiaUWTND9oKNcNM94StNStnuw,14cxMDpBNLsNEXWyCzked3zghzaYWXwoqGT4h12GqQXdVhmn,16idW666JZya7ZrGcL2nqSRZLRbQY3Tnf8HorbpMCeG83uq,19z6EGB9qgN7MsQACNYCmotAVb8FNL11JiXg7FV1hs6w3nc,13asdY4e7sWdJ4hbGW9n2rkNro1mx5YKB6WBCC9gvqKmLvNH,1qFBkRovtyKwWznGdQrA2qvmpndhM513TPoWf6f4oTVReLs,14N5nJ4oR4Wj36DsBcPLh1JqjvrM2Uf23No2yc2ojjCvSC24,1ejsVb4xGv2kVGeM2x9iGxJdvLE9QMn8Nh8dhMv8cHCHnTL,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
16Ak2D2ZApaR6sFKkawmC8ea93ASD4aB54eTMvEkDjCFZLjp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39498643450307
14FzsWSxgWMmXwafTiGf1YdXZgEoAY7dEv2Qr9h1sCHaWFt2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1084592814654
15LL8mQJQ7wsxrjxMyBxjphkNxciwirC6cQ5AprmmEt8KGNf,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,36219110590042
11888uchjZxbyc7RRstxZ8qepsskRjJhJatrxryBTdHuWfK,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2352067003888369
1ZQpD1knimbf4RLUyoipThPdhvEXJHud2sFDSe3cWPVN3q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3894263543601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16XMi2A7NvUuwt632GH7awa7P5MYJQrteJJ3iB13FCqeXPok,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2780000000000
16A5SmtCw1sTDr6zXdJ9hShqUX1XbYKXG1kk8Qv2BPhTsDNC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,245783340884
14N6uWVnUDZQfNf66HJsSFPLmJHm1p19Y44CBmVoBi9DDbRD,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,14529640


In [85]:
random_solution_amount = getAmount(random_solution)
random_solution_variance = getVariance(random_solution)
random_solution_assignment = getAssignment(random_solution)

random_solution_kpis = [random_solution_amount, random_solution_variance, random_solution_assignment]
random_solution_kpis

[992548013749737.1, -3.4153427308951364e+16, -69197]

In [86]:
random_solution2 = binary_matrix_dropped.sample(n=number_of_validators, axis=1)
random_solution2['amount'] = nominators.set_index('stash_address')['bonded_amount']
random_solution2_amount = getAmount(random_solution2)
random_solution2_variance = getVariance(random_solution2)
random_solution2_assignment = getAssignment(random_solution2)

random_solution_kpis2 = [random_solution2_amount, random_solution2_variance, random_solution2_assignment]
random_solution_kpis2

[908612962195794.6, -3.3359735635337332e+16, -139306]

In [87]:
score = calc_target(concatenateSolutions(random_solution_kpis, random_solution_kpis2))
score

-0.4713940828029838

In [88]:
score = calc_target(concatenateSolutions(random_solution_kpis2, random_solution_kpis))
score

4.957541674776994

In [89]:
validators["stash_address"]
ref_sol = binary_matrix[validators["stash_address"]]

ref_sol['amount'] = nominators.set_index('stash_address')['bonded_amount']
ref_sol_amount = getAmount(ref_sol)
ref_sol_variance = getVariance(ref_sol)
ref_sol_assignment = getAssignment(ref_sol)


/tmp/ipykernel_14137/3737593548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_sol['amount'] = nominators.set_index('stash_address')['bonded_amount']


In [90]:
ref_sol_kpis = [ref_sol_amount, ref_sol_variance, ref_sol_assignment]
score = calc_target(concatenateSolutions(ref_sol_kpis, random_solution_kpis))
score

27.01443564452544

In [91]:
ref_sol_kpis = [ref_sol_amount, ref_sol_variance, ref_sol_assignment]
score = calc_target(concatenateSolutions(ref_sol_kpis, random_solution_kpis2))
score

9.697103855161897

## Gready Solution

In [92]:
def solve(nominators, num_of_vals):
    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]
    selected_validators = set()

    binary_matrix.sort_values(by='amount', inplace=True, ascending=False)
    i = 0

    while len(selected_validators) < num_of_vals:
        cols = binary_matrix.columns[binary_matrix.iloc[i] == 1].tolist()
        to_add = num_of_vals - len(selected_validators)
        if len(cols) <= to_add:
            selected_validators.update(cols)
        else:
            selected_validators.update(cols[:to_add])
        i += 1

    result = binary_matrix[list(selected_validators)]
    result['amount'] = nominators.set_index('stash_address')['bonded_amount']

    return result

In [93]:
def solve_2(nominators, num_validators=10):
    nominators_copy = nominators.copy(deep=True)
    nominators_copy['targets'] = nominators_copy['targets'].apply(lambda x: x.split(','))
    
    expanded_nominators = nominators_copy.explode('targets')
    expanded_nominators['bonded_amount'] = expanded_nominators['bonded_amount'].astype(float)
    
    validator_stakes = expanded_nominators.groupby('targets')['bonded_amount'].sum().reset_index()
    validator_stakes = validator_stakes.rename(columns={'targets': 'validator_id', 'bonded_amount': 'total_stake'})
    validator_stakes_sorted = validator_stakes.sort_values(by='total_stake', ascending=False)

    selected_validators = []
    selected_nominators = set()

    for _, validator in validator_stakes_sorted.iterrows():
        validator_id = validator['validator_id']
        validator_nominators = set(expanded_nominators[expanded_nominators['targets'] == validator_id]['stash_address'])
        overlap = len(validator_nominators & selected_nominators)
        
        stakes = [v['total_stake'] for v in selected_validators] + [validator['total_stake']]
        variance = np.var(stakes)

        if len(selected_validators) < num_validators or (overlap < 2 and variance < np.var(stakes[:-1])):
            selected_validators.append(validator)
            selected_nominators.update(validator_nominators)
        
        if len(selected_validators) >= num_validators:
            break

    selected_validators_df = pd.DataFrame(selected_validators)
    
    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]

    return binary_matrix[list(selected_validators_df['validator_id']) + ['amount']]

In [94]:
greedy_solution = solve(nominators, number_of_validators)
greedy_solution

/tmp/ipykernel_14137/2417624070.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['amount'] = nominators.set_index('stash_address')['bonded_amount']


,13eKBARPFWBdXJAKg4fBTNUfcz4YAYfDTetRRApuz1kTDVDg,13WE4gVd4jW6o88Lnto9Y4WAhkMGeNNP7FWSqikeSCNqkwcv,12GTt3pfM3SjTU6UL6dQ3SMgMSvdw94PnRoF6osU6hPvxbUZ,14UieaEqDLiybuL5zHYribzkJGgEN8jJr77j9QdsJeaEUT1F,1CnnkaqNJWLeny9GMB2D66xQBbZGJBuPYsqdFzj3eGrMQRu,16LqQSto6QrpmKgqXWqc8HvpGvcY5tzo6wqQy97q7qQkKF3E,14xKzzU1ZYDnzFj7FgdtDAYSMJNARjDc2gNw4XAFDgr4uXgp,126psKUiNKidXPNJRkD9TYcZdruX4dymU3vsQ6g9GM4YtEis,12jjUEmaKuwC1SxQHe8eXpcv6AzdWiXdvf55VvP1y8bf548b,1YqYHy3dvtC4YGCSUnU67x3qgqCdo56XvYYhWV5LaERytST,...,14FYEWpz2JtBBNp2a2KHLr4d6NB3AJX16Zhr4KjAuoL2kxdR,14AvAa9VXp7iPh5Ga8ZEHMJKsfTjZcx7yRX28fcTEBxv9xKZ,16fL6kGX64fQ8cCvRu15idGS1VZnLiCZkkDWQer981ux5FRA,15hkz83sTMouD7GiiWP3SQc5DXaZFumYgJ6bf8Xq3dphPQ4h,15a9ScnYeVfQGL9HQtTn3nkUY1DTB8LzEX391yZvFRzJZ9V7,143XCiQMmvh7y6YB1MF3umY6Fy8Fzd36UgGb7teTxiEuvfmZ,15KJ8D2WRTYP2ea9PQTPt6sChC8ZjLnAeLu6DBQdbQbVftPw,138UfM4EToSwT2YoUAw3Dp6cVgEjQDPk75E4S43gsSAN1eNt,14yttAARJ6yJRzVBSvrqni3yNgjQ6hmewtC2fJkLTQQydmHD,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
15j4dg5GzsL1bw2U2AWgeyAk6QTxq43V7ZPbXdAmbVLjvDCK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,596162436600000001
12xtAYsRUrmbniiWQqJtECiBQrMn8AypQcXhnQAc6RB6XkLW,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,318000000000000000
14LPktAnKHGDy4aFz1rcuSq71FN5osbx1WSHPht2vmLdH74X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,273588274858046850
14Ns6kKbCoka3MS4Hn6b7oRw9fFejG8RH5rq5j63cWUfpPDJ,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,198720812199999999
12Z6FSvMFRjno881i1qpFndrtZtcVCLs5v4nR92McQNX31CT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,173400140000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13iSFVk1NBWUCUhzKSpu46YZ3wAGcCk5AJQzFJtwcN4EuNBK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4588471
126s2iN6AZeV8SbNs5A2rETep8i2QQPQuBPdRqKgGTWwYFkY,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3786819
1TbBMfqSfit5tp7vYtieTi1qUNhcR6D7yhniXFw9td5oX17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1747697


In [95]:
greedy_solution_kpis = calculateKpis(greedy_solution)
greedy_solution_kpis

[833711711650678.1, -1.4185189404932596e+16, -344439]

In [96]:
compareSolutions(ref_sol, greedy_solution)

1.642890683055002

In [100]:
greedy_solution = solve_2(nominators, number_of_validators)
greedy_solution_kpis = calculateKpis(greedy_solution)
compareSolutions(ref_sol, greedy_solution)

-0.0450881662089857